<a href="https://colab.research.google.com/github/22022658NguyenTienKhoi/13-Weather-Forecast/blob/main/regressors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [54]:
weather = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/hanoiweather.csv")

In [55]:
weather.set_index('datetime',inplace = True)

In [56]:
weather = weather[:'2010-01-01']

In [57]:
# Clean invalid columns
null_pct = weather.apply(pd.isnull).sum()/weather.shape[0]
valid_columns = weather.columns[null_pct < .725]
weather = weather.fillna(0)
weather = weather[valid_columns].copy()
weather.columns = weather.columns.str.lower()
weather.index = pd.to_datetime(weather.index)

In [58]:
# Setup time series for prophet
weather["y"] = weather.shift(-1)["tempmax"]
weather = weather.ffill()
weather["ds"] = weather.index

In [59]:
weather.drop(columns=['icon','description','stations','conditions','sunrise','sunset','name','preciptype'], inplace=True)

In [60]:
regressors = weather.columns[~weather.columns.isin(['y','ds'])]

In [61]:
regressors

Index(['tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover',
       'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility',
       'moonphase'],
      dtype='object')

In [62]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=2)
for train_index, test_index in tscv.split(weather):
    df_train, df_val = weather.iloc[train_index],weather.iloc[test_index]

In [63]:
'''m = Prophet()

# Initialize empty list to store selected regressors
selected_regressors = []

# Initialize variables to track best regressor set and performance
best_regressors = []
best_rmse = float('inf')

# Define function to evaluate Prophet model with given regressors
def evaluate_prophet_with_regressors(df, regressors):
    model = Prophet()
    for regressor in regressors:
        model.add_regressor(regressor)
    model.fit(df)

    # Assuming df contains historical time series data and you have a validation set df_val
    forecast = model.predict(df)
    y_true = df_val['y']
    y_pred = forecast['yhat'][-len(df_val):]

    return mean_squared_error(y_true, y_pred, squared=False)

# Greedy feature selection loop
for regressor in regressors:
    # Add the regressor to the selected set
    selected_regressors.append(regressor)

    # Evaluate model performance with the current selected regressors
    current_rmse = evaluate_prophet_with_regressors(df_train, selected_regressors)

    # Update best regressor set if the current one performs better
    if current_rmse < best_rmse:
        best_regressors = selected_regressors.copy()
        best_rmse = current_rmse
    else:
        # Remove the last added regressor if it doesn't improve performance
        selected_regressors.pop()

# Finalize the model with the best regressor set
for regressor in best_regressors:
    m.add_regressor(regressor)

# Fit the final model on the entire training dataset
m.fit(df_train)

# Make predictions on the validation dataset
forecast = m.predict(df_val)

# Evaluate the final model's performance
y_true = df_val['y']
y_pred = forecast['yhat'][-len(df_val):]
final_rmse = mean_squared_error(y_true, y_pred, squared=False)

print("Best regressors:", best_regressors)
print("Best RMSE:", best_rmse)
print("Final RMSE (using best regressor set on validation data):", final_rmse)'''

'm = Prophet()\n\n# Initialize empty list to store selected regressors\nselected_regressors = []\n\n# Initialize variables to track best regressor set and performance\nbest_regressors = []\nbest_rmse = float(\'inf\')\n\n# Define function to evaluate Prophet model with given regressors\ndef evaluate_prophet_with_regressors(df, regressors):\n    model = Prophet()\n    for regressor in regressors:\n        model.add_regressor(regressor)\n    model.fit(df)\n\n    # Assuming df contains historical time series data and you have a validation set df_val\n    forecast = model.predict(df)\n    y_true = df_val[\'y\']\n    y_pred = forecast[\'yhat\'][-len(df_val):]\n\n    return mean_squared_error(y_true, y_pred, squared=False)\n\n# Greedy feature selection loop\nfor regressor in regressors:\n    # Add the regressor to the selected set\n    selected_regressors.append(regressor)\n\n    # Evaluate model performance with the current selected regressors\n    current_rmse = evaluate_prophet_with_regres

In [64]:
import random
population_size = 20
num_generations = 20
mutation_rate = 0.1

regressors = list(regressors)

#Define evaluation
def evaluate_forecast(forecast):
    return np.mean((forecast['yhat'] - df_val['y'])**2)

# Define fitness function
def evaluate_fitness(regressors):
    # Initialize Prophet model
    model = Prophet()

    # Add selected regressors to the model
    for regressor in regressors:
        model.add_regressor(regressor)

    # Fit the model
    model.fit(df_train)  # Assuming df_train is your training data

    # Evaluate model performance on validation data
    forecast = model.predict(df_val)  # Assuming df_val is your validation data
    # Define your evaluation function and return fitness score
    fitness_score = evaluate_forecast(forecast)

    return fitness_score

# Initialize population with random regressor combinations
population = [random.sample(regressors, random.randint(1, len(regressors))) for _ in range(population_size)]

# Genetic Algorithm main loop
for generation in range(num_generations):
    # Evaluate fitness for each individual in the population
    fitness_scores = [evaluate_fitness(individual) for individual in population]

    # Select individuals for next generation based on fitness (tournament selection)
    selected_indices = random.sample(range(population_size), population_size // 2)
    selected_population = [population[i] for i in selected_indices]

    # Apply crossover to create new individuals
    new_population = []
    while len(new_population) < population_size:
        parent1, parent2 = random.choices(selected_population, k=2)
        crossover_point = random.randint(1, min(len(parent1), len(parent2)))
        child = parent1[:crossover_point] + parent2[crossover_point:]
        new_population.append(child)

    # Apply mutation to new individuals
    for i in range(population_size):
        if random.random() < mutation_rate:
            mutated_individual = random.sample(regressors, random.randint(1, len(regressors)))
            new_population[i] = mutated_individual

    # Update population with new individuals
    population = new_population

# Find the best regressor combination based on the final population's fitness
best_individual = max(population, key=evaluate_fitness)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpvz6cjjdz/ggtgf7pu.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpvz6cjjdz/ml5ze4c7.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=86663', 'data', 'file=/tmp/tmpvz6cjjdz/ggtgf7pu.json', 'init=/tmp/tmpvz6cjjdz/ml5ze4c7.json', 'output', 'file=/tmp/tmpvz6cjjdz/prophet_model40ve8e_9/prophet_model-20240508154337.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
15:43:37 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
15:43:38 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpvz

In [65]:
print(best_individual)

['precip', 'humidity', 'temp', 'cloudcover', 'feelslikemax', 'moonphase', 'precipprob', 'precip', 'tempmax', 'windspeed', 'dew', 'sealevelpressure', 'feelslikemin']
